In [1]:
from pystackreg import StackReg
import numpy as np
from skimage.external.tifffile import TiffFile 
from tifffile import memmap
import pandas as pd
import os, re
from PIL import Image
from joblib import Parallel, delayed

In [2]:
tiffs_directory = "/home/georgeos/Storage/Dropbox (Cambridge University)/DATA_Bakshi_PaulssonLab/40x_Ph2_testData/40x_Ph2_Test_1.5_ND2_extracted"
if tiffs_directory[-1] != "/":
    tiffs_directory += "/"
tiff_names = os.listdir(tiffs_directory)
tiff_names.sort()
experiment_file_data = []
for name in tiff_names:
    file_data = re.split(' |\_| |\.| ',name)[:3]
    file_data[1] = file_data[1][1:]
    file_data.append(tiffs_directory + name)
    experiment_file_data.append(file_data)
experiment_file_data = pd.DataFrame(experiment_file_data)
experiment_file_data.columns = ["position", "time", "channel", "directory"]

In [3]:
n_positions = len(experiment_file_data["position"].unique())
n_times = len(experiment_file_data["time"].unique())
n_channel = len(experiment_file_data["channel"].unique())
y_len, x_len = np.array(Image.open(experiment_file_data["directory"][0])).shape

In [4]:
positions = experiment_file_data["position"].unique()
times = experiment_file_data["time"].unique()
channels = experiment_file_data["channel"].unique()
reference_time = "0003"
reference_channel = "BF"

In [5]:
memmap_image = memmap("../test.tif",shape=(n_positions, n_channel, n_times, y_len, x_len), dtype=np.uint16)
def register_transform(memmap_image, time, reference_channel, reference_time):
    pos_val = 0
    for position in positions:
        channel_val = 0
        reference_image_directory = experiment_file_data[(experiment_file_data["position"] == position) 
                 & (experiment_file_data["time"] == reference_time)
                 & (experiment_file_data["channel"] == reference_channel)
                ]["directory"].iloc[0]
        reference_image = np.array(Image.open(reference_image_directory))
        image_for_register_ref_channel = experiment_file_data[(experiment_file_data["position"] == position) 
                                                & (experiment_file_data["channel"] == reference_channel)
                                                & (experiment_file_data["time"] == time)
                                               ]
        ref_channel_image_dir = image_for_register_ref_channel["directory"].iloc[0]
        current_ref_channel_image = np.array(Image.open(ref_channel_image_dir))
        sr = StackReg(StackReg.RIGID_BODY)
        sr.register(reference_image, current_ref_channel_image)
        
        for channel in channels:
            current_data = experiment_file_data[(experiment_file_data["position"] == position) 
                                                & (experiment_file_data["channel"] == channel)
                                                & (experiment_file_data["time"] == time)
                                               ]
            current_image_dir = current_data["directory"].iloc[0]
            current_image = np.array(Image.open(current_image_dir))
            out_tra = sr.transform(current_image)
            memmap_image[pos_val,channel_val,int(time),:,:] = out_tra
            channel_val += 1
        pos_val += 1

In [6]:
_ = Parallel(n_jobs=-1)(delayed(register_transform)(
    memmap_image,
    time,reference_channel,
    reference_time
) for time in times)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]